In [117]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [75]:
areaharvested = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC Area HarvestedRegionType_fulll.csv")
lucagri = pd.read_csv("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/luc_agri_prop_crop_region.csv")

In [76]:
areaharvested['NGFBFC'] = areaharvested['NGFBFC'].str.lower()
areaharvested.head()

,IMAGE Region Name,NGFBFC,time,value
0,Brazil,maize,1970,1.129294e+07
1,Brazil,maize,1975,1.208552e+07
2,Brazil,maize,1980,1.078206e+07
3,Brazil,maize,1985,1.065404e+07
4,Brazil,maize,1990,1.301961e+07


In [77]:
#1. a
lucagri_groupby = lucagri.groupby(['IMAGE Region Name', 'time', 'type']).sum()
lucagri_index = lucagri_groupby.reset_index()
lucagri_index['value'] = lucagri_index['value'] * 1E9
lucagri_index.head()

,IMAGE Region Name,time,type,value
0,Brazil,1975,grass,4.888251e+09
1,Brazil,1975,maize,4.190159e+08
2,Brazil,1975,oil & palm fruit,2.493429e+05
3,Brazil,1975,other non-food & luxury & spices,1.089310e+08
4,Brazil,1975,other temperate cereals,1.227032e+07


In [78]:
# 1. b c
df_copy = areaharvested.copy()
for region, time, ngfbfc in zip(df_copy["IMAGE Region Name"].to_list(), df_copy["time"].to_list(), df_copy["NGFBFC"].to_list()):
    try:
        data1 = areaharvested[(areaharvested['IMAGE Region Name'] == region) & (areaharvested['time'] == time) & (areaharvested['NGFBFC'] == ngfbfc)]['value'].to_numpy()
        data2 = lucagri_index[(lucagri_index['IMAGE Region Name'] == region) & (lucagri_index['time'] == time) & (lucagri_index['type'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_copy[(df_copy['IMAGE Region Name'] == region) & (df_copy['time'] == time) & (df_copy['NGFBFC'] == ngfbfc)].index
        df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
    except KeyError as e:
        print(f"Error: {e} for region {region}")

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: invalid value encountered in double_scalars
  df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  df_copy.loc[mask, 'CF'] =  data2[0] / data1[0]
C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_8068/2745533094.py:12: RuntimeWarning: divide by zero 

In [81]:
df_copy.head()

,IMAGE Region Name,NGFBFC,time,value,CF
0,Brazil,maize,1970,1.129294e+07,0.000000
1,Brazil,maize,1975,1.208552e+07,34.670896
2,Brazil,maize,1980,1.078206e+07,17.880961
3,Brazil,maize,1985,1.065404e+07,13.610512
4,Brazil,maize,1990,1.301961e+07,33.276575


In [79]:
# 2 c
df_copy2 = areaharvested.copy()
for time in range(1975, 2020, 5):
    mask = df_copy2[df_copy2['time'] == time].index # tahun berikutnya
    df_copy2.loc[mask, "selisih area harvested"] = np.where(areaharvested[(areaharvested['time'] == time)]['value'].to_numpy() > areaharvested[(areaharvested['time'] == time-5)]['value'].to_numpy(),
                            areaharvested[(areaharvested['time'] == time)]['value'].to_numpy() - areaharvested[(areaharvested['time'] == time-5)]['value'].to_numpy(), 0)  

df_copy2 = df_copy2[df_copy2['time'] > 1970]
df_copy2['selisih area harvested'] = df_copy2['selisih area harvested'].fillna(0)
df_copy2 = df_copy2[df_copy2['selisih area harvested'] > 0].drop(columns="value")
df_copy2.head()

,IMAGE Region Name,NGFBFC,time,selisih area harvested
1,Brazil,maize,1975,7.925840e+05
4,Brazil,maize,1990,2.365564e+06
5,Brazil,maize,1995,2.755675e+06
7,Brazil,maize,2005,2.043551e+06
8,Brazil,maize,2010,1.344975e+06


In [80]:
# 2 d
for region, time, ngfbfc in zip(df_copy2["IMAGE Region Name"].to_list(), df_copy2["time"].to_list(), df_copy2["NGFBFC"].to_list()):
    try:
        data1 = df_copy2[(df_copy2['IMAGE Region Name'] == region) & (df_copy2['time'] == time) & (df_copy2['NGFBFC'] == ngfbfc)]['selisih area harvested'].to_numpy()
        data2 = lucagri_index[(lucagri_index['IMAGE Region Name'] == region) & (lucagri_index['time'] == time) & (lucagri_index['type'] == ngfbfc)]['value'].to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        mask = df_copy2[(df_copy2['IMAGE Region Name'] == region) & (df_copy2['time'] == time) & (df_copy2['NGFBFC'] == ngfbfc)].index
        df_copy2.loc[mask, 'CF'] =  data2[0] / data1[0]
    except KeyError as e:
        print(f"Error: {e} for region {region}")

df_copy2.head()

,IMAGE Region Name,NGFBFC,time,selisih area harvested,CF
1,Brazil,maize,1975,7.925840e+05,528.670689
4,Brazil,maize,1990,2.365564e+06,183.147832
5,Brazil,maize,1995,2.755675e+06,427.028477
7,Brazil,maize,2005,2.043551e+06,116.510542
8,Brazil,maize,2010,1.344975e+06,19.269897


In [92]:
# 3 a
luc_agri_sum = pd.DataFrame({"IMAGE Region Name": df_copy["IMAGE Region Name"].to_list(), "NGFBFC": df_copy["NGFBFC"].to_list()})
luc_agri_sum = luc_agri_sum.drop_duplicates(subset=["IMAGE Region Name", "NGFBFC"])

for region in df_copy2["IMAGE Region Name"].unique():
    for ngfbfc in df_copy2["NGFBFC"].unique():
        mask = luc_agri_sum[(luc_agri_sum["NGFBFC"] == ngfbfc) & (luc_agri_sum["IMAGE Region Name"] == region)].index
        data1 = df_copy[(df_copy["NGFBFC"] == ngfbfc) & (df_copy["IMAGE Region Name"] == region)].groupby(['IMAGE Region Name', 'NGFBFC'])['CF'].sum().to_numpy()
        data2 = df_copy2[(df_copy2["NGFBFC"] == ngfbfc) & (df_copy2["IMAGE Region Name"] == region)].groupby(['IMAGE Region Name', 'NGFBFC'])['CF'].sum().to_numpy()
        if data1.size == 0:
            data1 = [0]
        if data2.size == 0: 
            data2 = [0]
        luc_agri_sum.loc[mask, 'total CF'] = data1[0] + data2[0]

luc_agri_sum   

,IMAGE Region Name,NGFBFC,total CF
0,Brazil,maize,1594.226465
11,Brazil,oil & palm fruit,1888.556239
22,Brazil,other non-food & luxury & spices,994.979277
33,Brazil,other temperate cereals,2183.455383
44,Brazil,plant based fibres,259.186292
...,...,...,...
4521,W. Europe,tropical cereals,46029.074526
4532,W. Europe,tropical oil crops,0.394656
4543,W. Europe,tropical roots & tubers,0.000000
4554,W. Europe,vegetables & fruits,0.000000


In [111]:
# 3 b
#jika punya 11 tahun buat rata" digroupby berdasarkan image region dan ngfbfc, lintas tahun

luc_agri_cf = pd.DataFrame({"IMAGE Region Name": df_copy2["IMAGE Region Name"].to_list(), "NGFBFC": df_copy2["NGFBFC"].to_list(), "time": df_copy2["time"].to_list()})

for region in df_copy2["IMAGE Region Name"].unique():
    for ngfbfc in df_copy2["NGFBFC"].unique():
        for time in df_copy2["time"].unique():
            mask = luc_agri_cf[(luc_agri_cf["IMAGE Region Name"] == region) & (luc_agri_cf["NGFBFC"] == ngfbfc) & (luc_agri_cf["time"] == time)].index

            data1 = df_copy[(df_copy["NGFBFC"] == ngfbfc) & (df_copy["IMAGE Region Name"] == region) & (df_copy["time"] == time)]['CF'].to_numpy()
            data2 = df_copy2[(df_copy2["NGFBFC"] == ngfbfc) & (df_copy2["IMAGE Region Name"] == region) & (df_copy2["time"] == time)]['CF'].to_numpy()
            
            if data1.size == 0:
                data1 = [0]
            if data2.size == 0: 
                data2 = [0]
            luc_agri_cf.loc[mask, 'CF'] = data1[0] + data2[0]

In [123]:
# 3 a
luc_crops_finall = pd.DataFrame({"IMAGE Region Name": df_copy["IMAGE Region Name"].to_list(), "NGFBFC": df_copy["NGFBFC"].to_list()})
luc_crops_finall = luc_crops_finall.drop_duplicates(subset=["IMAGE Region Name", "NGFBFC"])

for region in luc_crops_finall["IMAGE Region Name"].unique():
    for ngfbfc in luc_crops_finall["NGFBFC"].unique():
        mask = luc_crops_finall[(luc_crops_finall["NGFBFC"] == ngfbfc) & (luc_crops_finall["IMAGE Region Name"] == region)].index
        
        data1 = luc_agri_sum[(luc_crops_finall["NGFBFC"] == ngfbfc) & (luc_crops_finall["IMAGE Region Name"] == region)]['total CF'].to_numpy()
        data2 = len(luc_agri_cf[(luc_agri_cf['CF'] > 0) & (luc_agri_cf["NGFBFC"] == ngfbfc)  & (luc_agri_cf["IMAGE Region Name"] == region)].index)
        
        if data1.size == 0:
            data1 = [0]
            
        luc_crops_finall.loc[mask, 'average CF'] = data1[0]/data2

luc_crops_finall

,IMAGE Region Name,NGFBFC,average CF
0,Brazil,maize,2.657044e+02
11,Brazil,oil & palm fruit,2.098396e+02
22,Brazil,other non-food & luxury & spices,2.487448e+02
33,Brazil,other temperate cereals,2.729319e+02
44,Brazil,plant based fibres,2.591863e+02
...,...,...,...
4521,W. Europe,tropical cereals,4.602907e+04
4532,W. Europe,tropical oil crops,inf
4543,W. Europe,tropical roots & tubers,NaN
4554,W. Europe,vegetables & fruits,NaN
